In [1]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import pyproj
from scipy.interpolate import griddata
import cftime

In [4]:
home_path = os.path.expanduser("~")

path = '/DataFiles'
era = xr.open_dataset(home_path + path + "/ae3baa6a74f0aa315dc3de6f83298f0e.nc")

path2 = '/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - MacBook Air/Python/Ice Cores/data/model/ccsm4_last_millenium'


In [61]:
# pr has units m -> convert to mm / kg m^2
pr = era["tp"] * 1000


lats = era["latitude"].values
lons = era["longitude"].values

# Compute number of days in each month (xarray provides days_in_month)
days_in_month = pr["valid_time"].dt.days_in_month
seconds_in_month = days_in_month 

# Convert monthly mean rate (mm/s) → monthly total (mm)
pr_monthly_total = pr * seconds_in_month

# Restrict to 1979–2000
pr_sel = pr_monthly_total.sel(valid_time=slice("1979-01-01", "2000-12-31"))

# Group by year and sum over months
pr_annual = pr_sel.groupby("valid_time.year").sum("valid_time")

# pr_annual is now ERA5 (year, lat, lon) with units mm/year
print(pr_annual.shape)

(22, 301, 3600)


In [63]:
# Mapping the ice core locations to ERA5 indices
prCoords = pd.read_csv(home_path + path + "/AccumCoresCoords.csv", header=None)
coresLat = prCoords.iloc[:, 2].values
coresLonFirst = prCoords.iloc[:, 3].values
coresLon = (coresLonFirst + 360) % 360

lat_idx = []
lon_idx = []
grid_lats = []
grid_lons = []

for la, lo in zip(coresLat, coresLon):
    pt = era.sel(latitude=la, longitude=lo, method="nearest")

    # Get matched coordinates
    glat = float(pt.latitude.values)
    glon = float(pt.longitude.values)

    # Find indices in the original arrays
    i_lat = np.where(lats == glat)[0][0]
    i_lon = np.where(lons == glon)[0][0]

    grid_lats.append(glat)
    grid_lons.append(glon)
    lat_idx.append(i_lat)
    lon_idx.append(i_lon)

# --- Add results back to DataFrame ---
prCoords["lat_idx"] = lat_idx
prCoords["lon_idx"] = lon_idx
prCoords["grid_lat"] = grid_lats
prCoords["grid_lon"] = grid_lons

print(prCoords.head())

                       0                       1        2       3  lat_idx  \
0  vrs-13 (vostok stack)  Vostok composite VRS13 -78.4700  106.83      185   
1              B31 3.43W           B31Site DML07 -75.5800   -3.43      156   
2           B32 0.00667E           B32Site DML05 -75.0000   -0.01      150   
3            B33 6.4983E           B33Site DML17 -75.1700    6.50      152   
4              FB96DML01               FB96DML01 -74.8583   -2.55      149   

   lon_idx  grid_lat  grid_lon  
0     2868     -78.5     106.8  
1     3599     -75.6     179.9  
2     3599     -75.0     179.9  
3     1865     -75.2       6.5  
4     3599     -74.9     179.9  


In [64]:
# The ice core accumulation records
accumData = pd.read_csv(home_path + path + "/AccumCoresData.csv", header=None)
print(accumData)

              0                      1          2             3            4   \
0    Site number                      1          2             3            4   
1           Year  vrs-13 (vostok stack)  B31 3.43W  B32 0.00667E  B33 6.4983E   
2           2020                    NaN        NaN           NaN          NaN   
3           2019                    NaN        NaN           NaN          NaN   
4           2018                    NaN        NaN           NaN          NaN   
..           ...                    ...        ...           ...          ...   
218         1804                  39.93       69.6          69.5         31.7   
219         1803                  17.04       75.7            78         41.1   
220         1802                  26.64       66.2          63.3         45.1   
221         1801                   5.79         52          62.9         44.6   
222         1800                  30.37         79          80.8         45.7   

            5          6   

In [65]:
# The ERA5 metrics
era5Metrics = np.array(pd.read_csv(home_path + path + '/ERA5Metrics.csv'))

era5_accum_ais = era5Metrics[:, 1]
era5_accum_wais = era5Metrics[:, 2]
era5_accum_eais = era5Metrics[:, 3]
era5_temp_ais = era5Metrics[:, 4]
era5_temp_wais = era5Metrics[:, 5]
era5_temp_eais = era5Metrics[:, 6]

In [66]:
accumCores = accumData.iloc[2:, 1:][
    (accumData.iloc[2:, 0].astype(int) >= 1979) & (accumData.iloc[2:, 0].astype(int) <= 2000)
].to_numpy()[::-1]

accumCoordsInd = prCoords.iloc[:, 4:6].to_numpy()

print(accumCores.shape)
print(accumCoordsInd.shape)

(22, 85)
(85, 2)


In [67]:
# Performing the proxy/local regression
n_years, n_lat, n_lon = pr_annual.shape
n_cores = accumCores.shape[1]

# Initialize array to store local ERA5 time series at proxy locations
localERA5 = np.zeros((n_years, n_cores))

for i in range(n_cores):
    lat_idx, lon_idx = accumCoordsInd[i]
    localERA5[:, i] = pr_annual[:, lat_idx, lon_idx]